
## Google
# Applied Contextual Bandits for classification problems using Tensorflow and bigquery

**Authors:**  <br>
Anant Nawalgaria<br>
Alex Erfurt

Machine Learning Specialists






# Table of Contents

1. [Initial Setup: installing and importing required libraries](#1.-Initial-Setup:-installing-and-importing-required-libraries)
2. [Loading the training dataset from Bigquery to Tensorflow](#2.-Loading-the-training-dataset-from-Bigquery-to-Tensorflow)
3. [Initializing and configuring the Environment](#3.-Initializing-and-configuring-the-Environment)
4. [Initializing the Agent](#4.-Initializing-the-Agent)
5. [Define and link the evaluation metrics](#5.-Define-and-link-the-evaluation-metrics)
6. [Initialize & configure the Replay Buffer](#6.Initialize-&-configure-the-Replay-Buffer)
7. [Setup and Train the model](#7.-Setup-and-train-the-Model)
8. [Inferencing with trained model & Tensorboard Evaluation](#8.-Inferencing-with-trained-model-&-Tensorboard-evaluation)

## Introduction


Multi-Armed Bandit (MAB) is a Machine Learning framework in which an agent has to select actions (arms) in order to maximize its cumulative reward in the long term. In each round, the agent receives some information about the current state (context), then it chooses an action based on this information and the experience gathered in previous rounds. At the end of each round, the agent receives the reward assiociated with the chosen action.


, https://www.tensorflow.org/agents/tutorials/intro_bandit#multi-armed_bandits_and_reinforcement_learning
    

## 1. Initial Setup: installing and importing required Libraries

In [ ]:
!pip install --quiet --upgrade --force-reinstall tensorflow==2.3 tensorflow_probability tensorflow-io --use-feature=2020-resolver
!pip install tf_agents gast==0.3.3 --upgrade --use-feature=2020-resolver

In [ ]:
import os
import numpy as np
import functools
import os

import numpy as np

import tensorflow as tf  # pylint: disable=g-explicit-tensorflow-version-import
import tensorflow_probability as tfp
from tf_agents.bandits.agents import lin_ucb_agent
from tf_agents.bandits.agents import linear_thompson_sampling_agent as lin_ts_agent
from tf_agents.bandits.agents import neural_epsilon_greedy_agent as eps_greedy_agent
from tf_agents.bandits.agents.examples.v2 import trainer
from tf_agents.bandits.environments import classification_environment as ce
from tf_agents.bandits.environments import environment_utilities as env_util
from tf_agents.bandits.metrics import tf_metrics as tf_bandit_metrics
from tf_agents.networks import q_network
from tf_agents.drivers import dynamic_step_driver
from tf_agents.eval import metric_utils
from tf_agents.metrics import tf_metrics
from tf_agents.policies import policy_saver
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.trajectories import time_step as ts
tfd = tfp.distributions

In [ ]:
#!gsutil cp covtype.data gs://test_processig/
#!gsutil cp Training notebook-Copy1.ipynb gs://test_processig/

## 2. Loading the training dataset from Bigquery to Tensorflow

If you are not already familiar with tf.Dataset, it is recommended to take a few minutes to familiarize yourself with how it works.
Some reference material you can find [here](https://www.tensorflow.org/api_docs/python/tf/data/Dataset). Tensorflow_io offers you to connect and stream data directly from bigquery: obviating any need to write custom data retrieval and processing routines. You can read about the Tensorflow Bigquery connector [here](https://github.com/tensorflow/io/tree/v0.15.0/tensorflow_io/bigquery). 
In short we will first initialize a bigqueryclient pointing it to the data source, then make a tf.Dataset out of which using  reads data from parallel from bigquery.parallel_read_rows().

In [ ]:
ROOT_DIR = "/home/jupyter/tmp/quick_test/v7/"

BATCH_SIZE = 128
TRAINING_LOOPS = 10
STEPS_PER_LOOP = 2
AGENT_ALPHA = 10.0

EPSILON = 0.01
LAYERS = (300, 200, 100, 100, 50, 50)
LR = 0.002

AGENT_CHECKPOINT_NAME = 'agent'
STEP_CHECKPOINT_NAME = 'step'
CHECKPOINT_FILE_PREFIX = 'ckpt'

In [ ]:
#sol
from tensorflow.python.framework import dtypes
from tensorflow_io.bigquery import BigQueryClient
from tensorflow_io.bigquery import BigQueryReadSession

def features_and_labels(features):
  label = features.pop('int64_field_54') # this is what we will train for
  
  return tf.cast(tf.stack(tf.nest.flatten(features), axis=0), tf.float32), tf.cast(label-1, tf.int32)

#COL_NAMES = ['Time', 'Amount', 'Class'] + ['V{}'.format(i) for i in range(1,29)]
#COL_TYPES = [dtypes.float64, dtypes.float64, dtypes.int64] + [dtypes.float64 for i in range(1,29)]

client = BigQueryClient()
num_samples = 581012

DATASET_GCP_PROJECT_ID, DATASET_ID, TABLE_ID,  = 'rl-internal-course.training_rl.covtype'.split('.')

selected_fields =["int64_field_{}".format(i) for i in range(55)]
output_types =  [ dtypes.int64 for field in selected_fields ]

bqsession = client.read_session(
    "projects/" + DATASET_GCP_PROJECT_ID,
    DATASET_GCP_PROJECT_ID, TABLE_ID, DATASET_ID,
    selected_fields, output_types)

tf_dataset = bqsession.parallel_read_rows(block_length = num_samples, num_parallel_calls=tf.data.experimental.AUTOTUNE).map(features_and_labels).repeat().shuffle(buffer_size=400000)


In [ ]:
tf_dataset

## 3. Initializing and configuring the Environment

An environment in the TF-Agents Bandits library is a class that provides observations and reports rewards based on obseravtions and actions.
In this section we instantiate the "covertype bandit environment". Originally, the covertype dataset is a set of labeled examples for different types of forests, taken from here: https://archive.ics.uci.edu/ml/datasets/covertype. To cite the source:

"Predicting forest cover type from cartographic variables only (no remotely sensed data). The actual forest cover type for a given observation (30 x 30 meter cell) was determined from US Forest Service (USFS) Region 2 Resource Information System (RIS) data. Independent variables were derived from data originally obtained from US Geological Survey (USGS) and USFS data."

In the TF-Agents bandits library, there is an environment wrapper (named ClassificationBanditEnvironment) that can turn any multiclass labeled dataset to a bandit environment. The context (or observation) will be the features in the dataset, the actions are the label classes, and the rewards are calculated based on some stochastic function of the actual and the guessed labels. This latter function is defined by a table of distributions. For our covertype example, this table is simply the deterministic identity matrix:

In [ ]:
# initialize the distribution
covertype_reward_distribution = tfd.Independent(
    tfd.Deterministic(tf.eye(7)), reinterpreted_batch_ndims=2)


In [ ]:
covertype_reward_distribution.sample()

# provides an interface to return a reward given an action for features, tf_dataset provides labels

In [ ]:
# Initializing the Classification Bandit Environment with the dataset, and reward distri ution
environment = ce.ClassificationBanditEnvironment(
    tf_dataset, covertype_reward_distribution, BATCH_SIZE)

In [ ]:
# 
environment.reward_spec()

## 4. Initializing the Agent
Now that we have the environment and metrics intialized from the Tf.dataset loaded from big query we reach the part where we define and initialize our policy and the Agent which will be our utilize that policy to make decisions given an observation. We have several policies: as shown here:

   1. [NeuralEpsilonGreedyAgent](https://medium.com/analytics-vidhya/the-epsilon-greedy-algorithm-for-reinforcement-learning-5fe6f96dc870): The neural episilon greed algorithm makes a value estimate for all the arms, and then chooses the best arm with the probaility (1-episilon) and any of the random arms with a probability of epsilon. this balances the exploration-exploitation tradeoff and epsilon is set to a small value like 10%. Example: In this example we have seven arms: one of each of the classes, and if we set episilon to say 10%, then 90% of the times the agent will choose the arm with the highest value estimate ( expplotiing the one most likely to be the predicted class) and 10% of the time it will choose a random arm from all of the 7 arms( thus exploring the other possibilities). Refer [here](https://www.tensorflow.org/agents/api_docs/python/tf_agents/bandits/agents/neural_epsilon_greedy_agent/NeuralEpsilonGreedyAgent) for more information of the tensorflow agents version of the same.
   
   Each Agent is initiatlizied with a policy: which is essentially the function approximator ( be it linear or non linear) for estimating the Q values. Ther agen uses this policy, adds the exploration-exploitation component on top of this and is then trained. In this example we will use a Deep Q Network as our policy


In [ ]:
network = q_network.QNetwork(
          input_tensor_spec=environment.time_step_spec().observation,
          action_spec=environment.action_spec(),
          fc_layer_params=LAYERS)

agent = eps_greedy_agent.NeuralEpsilonGreedyAgent(
  time_step_spec=environment.time_step_spec(),
  action_spec=environment.action_spec(),
  reward_network=network,
  optimizer=tf.compat.v1.train.AdamOptimizer(learning_rate=LR),
  epsilon=EPSILON)

## 5. Define and link the evaluation metrics


Just like you have metrics like accuracy/recall in supervised learning, in bandits we use the [regret](https://www.tensorflow.org/agents/tutorials/bandits_tutorial#regret_metric) metric per episode. To calculate the regret, we need to know what the highest possible expected reward is in every time step. For that, we define the `optimal_reward_fn`.

Another similar metric is the number of times a suboptimal action was chosen. That requires the definition if the `optimal_action_fn`.

In [ ]:
optimal_reward_fn = functools.partial(
    env_util.compute_optimal_reward_with_classification_environment,
    environment=environment)

optimal_action_fn = functools.partial(
    env_util.compute_optimal_action_with_classification_environment,
    environment=environment)

In [ ]:
regret_metric = tf_bandit_metrics.RegretMetric(optimal_reward_fn)
suboptimal_arms_metric = tf_bandit_metrics.SuboptimalArmsMetric(
    optimal_action_fn)

In [ ]:

step_metric = tf_metrics.EnvironmentSteps()
metrics = [tf_metrics.NumberOfEpisodes(),  #equivalent to number of steps in bandits problem
           regret_metric,  # measures regret
           suboptimal_arms_metric,  # number of times the suboptimal arms are pulled
           tf_metrics.AverageReturnMetric(batch_size=environment.batch_size)  # the average return
           ]



## 6. Initialize & configure the Replay Buffer
Reinforcement learning algorithms use replay buffers to store trajectories of experience when executing a policy in an environment. During training, replay buffers are queried for a subset of the trajectories (either a sequential subset or a sample) to "replay" the agent's experience. Sampling from the replay buffer facilitate data re-use and breaks harmful co-relation between sequential data in RL, although in contextual bandits this isn't absolutely required but still helpful.

The replay buffer exposes several functions which allow you to manipulate the replay buffer in several ways. Read more on them [here] (https://www.tensorflow.org/agents/tutorials/5_replay_buffers_tutorial)

In this demo we would be using the TFUniformReplayBuffer for which we need to initialize the buffer spec with the spec of the trajectory of the agent's policy, a chosen batch size( number of trajectories to store), and the maximum length of the trajectory. ( this is the amount of sequential time steps which will be considered as one data point). so a batch of 3 with 2 time steps each would result in a tensor of shape (3,2). Since unlike regular RL problems, Contextual bandits have only one time step we can keep max_length =1, however since this tutorial is to enable you for RL problems as well, let set it to 2. Do not worry, any contextual bandit agent will internally
split the time steps inside each data point such that the effective batch size ends up being (6,1). 

Create a Tensorflow based UniformReplayBuffer And initialize it with an appropriate values.
Recommended:
    Batch size= 128
    Max length = 2 ( 2 time steps per item)

In [ ]:
# solution
buf = tf_uniform_replay_buffer.TFUniformReplayBuffer(
      data_spec=agent.policy.trajectory_spec,
      batch_size=BATCH_SIZE,
      max_length=STEPS_PER_LOOP)



Now we have a Replay buffer but we also need something to fill it with. Often a common practice is to have 
the agent Interact with and collect experience with the environment, without actually learning from it ( i.e. only forward pass). This loop can  be either by you manually as shown [here](https://www.tensorflow.org/agents/tutorials/6_reinforce_tutorial#training_the_agent) or you can do it using the DynamicStepDriver.
The data encountered by the driver at each step is saved in a named tuple called Trajectory and broadcast to a set of observers such as replay buffers and metrics. 
This Trajectory includes the observation from the environment, the action recommended by the policy, the reward obtained, the type of the current and the next step, etc. 

In order for the driver to fill the replay buffer with data, as well as to compute ongoing metrics, it needs acess to the add_batch, funcitonality of the buffer, and the metrics ( both step and regular). Refer [here](https://www.tensorflow.org/agents/tutorials/5_replay_buffers_tutorial#data_collection) for more information aand example code on how initialize a step driver with observers. 


In [ ]:
# solution
replay_observer = [buf.add_batch, step_metric] + metrics  #  UNCOMMENT THIS BACK WHEN DEBUGGING IS DONE (GABOR)
# replay_observer = [buf.add_batch]  # Gabor's debug

driver = dynamic_step_driver.DynamicStepDriver(
      env=environment,
      policy=agent.collect_policy,
      num_steps=STEPS_PER_LOOP * environment.batch_size,
      observers=replay_observer )


 Here we provide you a helper function in order to save your agent, the metrics and its lighter policy seperately, while training the model. We make all the aspects into trackable objects and then use checkpoint to save as well warm restart a previous training. For more information on checkpoints and policy savers ( which will be used in the training loop below) refer [here](https://www.tensorflow.org/agents/tutorials/10_checkpointer_policysaver_tutorial)

In [ ]:
def restore_and_get_checkpoint_manager(root_dir, agent, metrics, step_metric):
  """Restores from `root_dir` and returns a function that writes checkpoints."""
  trackable_objects = {metric.name: metric for metric in metrics}
  trackable_objects[AGENT_CHECKPOINT_NAME] = agent
  trackable_objects[STEP_CHECKPOINT_NAME] = step_metric
  checkpoint = tf.train.Checkpoint(**trackable_objects)
  checkpoint_manager = tf.train.CheckpointManager(checkpoint=checkpoint,
                                                  directory=root_dir,
                                                  max_to_keep=5)
  latest = checkpoint_manager.latest_checkpoint

  if latest is not None:
    print('Restoring checkpoint from %s.', latest)
    checkpoint.restore(latest)
    print('Successfully restored to step %s.', step_metric.result())
  else:
    print('Did not find a pre-existing checkpoint. '
                 'Starting from scratch.')
  return checkpoint_manager


In [ ]:
checkpoint_manager = restore_and_get_checkpoint_manager(
  ROOT_DIR, agent, metrics, step_metric)
# saver = policy_saver.PolicySaver(agent.policy)
summary_writer = tf.summary.create_file_writer(ROOT_DIR)
summary_writer.set_as_default()

Now we have all the components ready to start training the model. Here is the process for Training the model
1. We first use the DynamicStepdriver instance to collect experience( trajectories) from the environment and fill up the replay buffer.
2. We then extract all the stored experience from the replay buffer by specfiying the batch size and num_steps the same as we initialized the driver with. We extract it as tf.dataset instance.
3. We then iterate on the tf.dataset and the first sample we draw actually has all the data batch_size*num_time_steps
4. the agent then trains on the acquired experience
5. the replay buffer is cleared to make space for new data
6. Log the metrics and store them on disk
7. Save the Agent ( via checkpoints) as well as the policy

In [ ]:
#solution
import warnings
warnings.filterwarnings('ignore')
TRAINING_LOOPS = 15000

for _ in range(TRAINING_LOOPS):
    driver.run()
    batch_size = driver.env.batch_size
    
    dataset = buf.as_dataset(
      
        sample_batch_size = BATCH_SIZE,
        num_steps=STEPS_PER_LOOP,
        single_deterministic_pass=True)

    experience, unused_info = next(iter(dataset))
    train_loss = agent.train(experience).loss
    buf.clear()
    metric_utils.log_metrics(metrics)
        # for m in metrics:
        # print(m.name, ": ", m.result())
    for metric in metrics:
        metric.tf_summaries(train_step=step_metric.result())
    checkpoint_manager.save()
    
    # saver.save(os.path.join(ROOT_DIR, "./", 'policy_%d' % step_metric.result()))

Now that our model is trained, what if we want to determine which action to take given a new "context": for that we will iterate on our dataset to get the next item,
    make a timestep out of it by wrapping the results using ts.Timestep. It expects step_type, reward, discount, and observation as input: since we are performing prediction you can fill 
        in dummy values for the first 3: only the observation/context is relevant. Read about how it works [here](https://www.tensorflow.org/agents/api_docs/python/tf_agents/trajectories/time_step/TimeStep), and perform the task below
        
       

In [ ]:
feature, label = iter(tf_dataset).next()

In [ ]:
step = ts.TimeStep(
        tf.constant(
            ts.StepType.FIRST, dtype=tf.int32, shape=[1],
            name='step_type'),
        tf.constant(0.0, dtype=tf.float32, shape=[1], name='reward'),
        tf.constant(1.0, dtype=tf.float32, shape=[1], name='discount'),
        tf.constant(feature,
                    dtype=tf.float32, shape=[1,54],
                    name='observation'))

agent.policy.action(step).action.numpy()

One final task : let us upload the tensoboard logs, to get an overview of the performance of our model. We will upload our logs to tensorboard.dev and for that you need to 
copy the following command in terminal and execute it from there, it will give you a link from which you need to copy/paste the authentication code, and once that is done, you will receive the 
url of your model evaluation, hosted on a public [tensorboard.dev](https://tensorboard.dev/) instance

In [ ]:
!tensorboard dev upload --logdir /home/jupyter/tmp/quick_test/v7/ --name "(optional) My latest experiment" --description "(optional) Agent trained"